In [ ]:
import numpy as np
from scipy import optimize, stats
from sklearn.neighbors import KernelDensity
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white', palette='Set2')
import ray
ray.init()

In [ ]:
df = pd.read_csv('data.csv', index_col=[0,1])

In [ ]:
lst_date = sorted(list(set(df['date.1'])))
ndate = len(lst_date)

# lst_stock = sorted(list(set(df.loc['2016-02-16']['Name.1'])))
set_stock = set(df.loc['2013-02-11']['Name.1'])
for i in lst_date:
    set_stock = set_stock & set(df.loc[i]['Name.1'])
lst_stock = sorted(list(set_stock))
nstock = len(lst_stock)

In [ ]:
plt.plot(df.swaplevel().loc['AAPL']['return'].values)
plt.legend(['AAPL'])
plt.show()

In [ ]:
plt.hist(df.swaplevel().loc['AAPL']['return'].values, bins=100)
plt.legend(['AAPL'])
plt.show()

utils

In [ ]:
def ratio2num(p, w):
    # from ratio to number, a good rounding function
    # print(w)
    assert np.abs(w.sum() - 1) < 1e-5 and len(w.shape) == 1
    L = w.shape[0]
    nums = p * w
    rnums = np.round(nums)

    if rnums.sum() > p:
        idx = np.argsort(rnums - nums)[::-1][:int(rnums.sum() - p)]
        rnums[idx] = rnums[idx] - 1
    elif rnums.sum() < p:
        idx = np.argsort(nums - rnums)[::-1][:int(p - rnums.sum())]
        rnums[idx] = rnums[idx] + 1

    return rnums

In [ ]:
suitable_pairs = {
    1: [[1]],
    2: [[2,0],[0,1]],
    3: [[3,0,0],[1,1,0],[0,0,1]],
    4: [[4,0,0,0],[2,1,0,0],[1,0,1,0],[0,0,0,1]],
    5: [[5,0,0,0,0],[3,1,0,0,0],[2,0,1,0,0],[1,2,0,0,0],[1,0,0,1,0],[0,1,1,0,0],[0,0,0,0,1]],
    6: [[6,0,0,0,0,0],[4,1,0,0,0,0],[3,0,1,0,0,0],[2,2,0,0,0,0],[2,0,0,1,0,0],[1,1,1,0,0,0],[1,0,0,0,1,0],[0,3,0,0,0,0],[0,1,0,1,0,0],[0,0,2,0,0,0],[0,0,0,0,0,1]],
    7: [[7,0,0,0,0,0,0],[5,1,0,0,0,0,0],[4,0,1,0,0,0,0],[3,2,0,0,0,0,0],[3,0,0,1,0,0,0],[2,1,1,0,0,0,0],[2,0,0,0,1,0,0],[1,3,0,0,0,0,0],[1,1,0,1,0,0,0],[1,0,2,0,0,0,0],[1,0,0,0,0,1,0],[0,2,1,0,0,0,0],[0,1,0,0,1,0,0],[0,0,1,1,0,0,0],[0,0,0,0,0,0,1]]
}

In [ ]:
def icd(t, alpha):
    # inverse cubic density
    c = 2 * (1 - alpha)**2
    a = 2 * alpha - 1
    h = (t >= alpha) * c / (t - a)**3
    return h

In [ ]:
def icd_qtile(q, alpha):
    # quantile of inverse cubic density
    c = 2 * (1 - alpha)**2
    a = 2 * alpha - 1
    return a + np.sqrt(c / 2 / (1 - q))

In [ ]:
def intg(s, alpha):
    c = 2 * (1 - alpha)**2
    a = 2 * alpha - 1
    sa1 = 1 + s * a
    intg = c * s / sa1**3 * np.log((1 - alpha) / (alpha + 1 / s)) + 2 * (1 - alpha) / sa1**2 + a / sa1
    return intg

In [ ]:
def lsd_icd(n, p, x, alpha, niter=50000):
    # LSD corresponding to an inverse cubic density
    eps = 1e-6
    z = x + eps * 1j

    # optimization
    opt = optimize.root(lambda s: s * (p / n * intg(s, alpha) - z) - 1, x0=np.ones_like(z) * 0.01j, method='krylov')
    s, flag = opt.x, opt.success
    
    # fixed point iteration
    # s = np.ones_like(z) * 1j
    # for _ in range(niter):
    #     s = 1 / (p / n * intg(s, alpha) - z)

    f = s.imag * n / p / np.pi
    return f

spectrum correction

In [ ]:
def spec_mom(n, p, L, eig_vals):
    y = p / n
    k = 2 * L - 1

    theta0 = np.concatenate([np.ones(L - 1) / L, (np.ones(L) + np.clip(np.random.randn(L), 0, 1)) * np.exp(np.log(eig_vals).mean())]) # first L-1 is w_1 ... w_{L-1}, last L is \lambda_1 ... \lambda_L
    fwl = lambda theta: 1 - theta[:L-1].sum() # w_L
    fbeta = lambda theta: np.array([theta[:L-1] @ theta[L-1:2*L-2]**i + fwl(theta) * theta[-1]**i for i in range(1, k + 1)])
    falphaj = lambda beta, j: np.sum([y**(p.sum() - 1) * np.exp(p @ np.log(beta)) * np.math.factorial(j) / np.math.factorial(j + 1 - p.sum()) / np.prod(list(map(np.math.factorial, p))) for p in np.array(suitable_pairs[j])])
    falpha = lambda beta: np.array([falphaj(beta[:j], j) for j in range(1, k + 1)])

    alpha = np.array([np.mean(eig_vals**i) for i in range(1, k + 1)])
    opt = optimize.minimize(lambda theta: 1e5 * np.sum((falpha(fbeta(theta)) - alpha)**2), theta0)
    theta, flag = opt.x, opt.success
    
    return np.concatenate([theta[:L-1], [fwl(theta)]]), theta[L-1:]
    

In [ ]:
def spec_lse(n, p, eig_vals):
    # in this project, we consider inverse cubic density for H(t), parametrized by \alpha
    unet = np.linspace(-10, 0, 50, endpoint=False)
    sn = np.zeros_like(unet)
    for i in range(unet.shape[0]):
        sn[i] = - (1 - p / n) / unet[i] + np.sum(1 / (eig_vals - unet[i])) / n
    
    def hat_unet(alpha):
        c = 2 * (1 - alpha)**2
        a = 2 * alpha - 1
        sa1 = 1 + sn * a
        intg = c * sn / sa1**3 * np.log((1 - alpha) / (alpha + 1 / sn)) + 2 * (1 - alpha) / sa1**2 + a / sa1
        return - 1 / sn + p / n * intg
    
    cons = ({'type': 'ineq', 'fun': lambda x: x - 1e-10}, {'type': 'ineq', 'fun': lambda x: 1 - 1e-10 - x})
    opt = optimize.minimize(lambda alpha: np.sum((unet - hat_unet(alpha))**2), x0=0.5, constraints=cons)
    alpha, flag = opt.x, opt.success
    
    return alpha

equal, plug-in, bootstrap enhancement, spectrum-corrected estimator

In [ ]:
def equal(n, p, sigma0, seed=1234):
    if seed != None:
        np.random.seed(seed)
    dates = lst_date[-n:]
    stocks = np.random.choice(lst_stock, p, replace=False)
    df_new = df.loc[dates].swaplevel().loc[stocks].swaplevel()['return']
    
    x = np.array(df_new).reshape(p, n) # p*n array
    bar_x = x.mean(axis=1)
    demean_x = x - bar_x.reshape(p, 1)
    bar_S = demean_x @ demean_x.T / n

    hat_c = np.ones(p) / p
    hat_R = hat_c @ bar_x
    hat_r = hat_c @ (bar_S @ hat_c)

    return bar_x, bar_S, hat_c, hat_R, hat_r

In [ ]:
def plugin(n, p, sigma0, x=None, seed=1234):
    if seed != None:
        np.random.seed(seed)
    dates = lst_date[-n:]
    stocks = np.random.choice(lst_stock, p, replace=False)
    df_new = df.loc[dates].swaplevel().loc[stocks].swaplevel()['return']

    if type(x) != np.ndarray: # x can be given as bootstrap samples
        x = np.array(df_new).reshape(p, n) # p*n array
    bar_x = x.mean(axis=1)
    demean_x = x - bar_x.reshape(p, 1)
    bar_S = demean_x @ demean_x.T / n
    
    ones = np.ones(p)
    S_inv_x = np.linalg.solve(bar_S, bar_x)
    S_inv_1 = np.linalg.solve(bar_S, ones)
    if sigma0 * ones @ S_inv_x < np.sqrt(bar_x @ S_inv_x):
        hat_c = sigma0 * S_inv_x / np.sqrt(bar_x @ S_inv_x)
    else:
        hat_b = np.sqrt((sigma0**2 * ones @ S_inv_1 - 1) / ((bar_x @ S_inv_x) * (ones @ S_inv_1) - (ones @ S_inv_x)**2))
        hat_c = S_inv_1 / (ones @ S_inv_1) + hat_b * (S_inv_x - (ones @ S_inv_x)/(ones @ S_inv_1) * S_inv_1)

    hat_R = hat_c @ bar_x
    hat_r = hat_c @ (bar_S @ hat_c)

    return bar_x, bar_S, hat_c, hat_R, hat_r

In [ ]:
def bootstrap(n, p, sigma0, repeats=30, seed=1234):
    bar_x, bar_S, cp, Rp, _ = plugin(n, p, sigma0, seed=seed) # plug-in estimator
    
    cs, Rs = [], []
    for _ in range(repeats):
        bsample = np.random.multivariate_normal(mean=bar_x, cov=bar_S, size=n).T # p*n 
        _, _, cb, Rb, _ = plugin(n, p, sigma0, x=bsample, seed=None)
        cs.append(cb)
        Rs.append(Rb)
    
    gamma = 1 / (1 - p / n)
    hat_c = cp + (cp - np.array(cs).mean(axis=0)) / np.sqrt(gamma)
    hat_R = Rp + (Rp - np.mean(Rs)) / np.sqrt(gamma)
    hat_r = hat_c @ (bar_S @ hat_c)

    return bar_x, bar_S, hat_c, hat_R, hat_r

In [ ]:
def spectrum(n, p, sigma0, method='lse', nspike=0, alpha=None, seed=1234):
    bar_x, bar_S, cp, Rp, _ = plugin(n, p, sigma0, seed=seed) # plug-in estimator

    # spectral decomposition
    std = np.sqrt(np.diag(bar_S))
    corr = np.diag(1 / std) @ bar_S @ np.diag(1 / std)
    eig_vals, eig_vecs = np.linalg.eig(corr)
    idx = eig_vals.argsort() # from small to large
    eig_vals, eig_vecs = eig_vals[idx], eig_vecs[:, idx]

    # spectrum correction
    if method == 'lse': # use inverse cubic density
        if alpha == None:
            alpha = spec_lse(n, p, eig_vals) if nspike == 0 else spec_lse(n, p, eig_vals[:-nspike])
        # print(alpha)
        spec = icd_qtile(np.linspace(1 / 2 / p, 1 - 1 / 2 / p, p), alpha)
    else:
        raise NotImplementedError
    
    hat_Sigma = np.diag(std) @ eig_vecs @ np.diag(spec) @ eig_vecs.T @ np.diag(std) # corrected estimate of covariance

    ones = np.ones(p)
    S_inv_x = np.linalg.solve(hat_Sigma, bar_x)
    S_inv_1 = np.linalg.solve(hat_Sigma, ones)
    if sigma0 * ones @ S_inv_x < np.sqrt(bar_x @ S_inv_x):
        hat_c = sigma0 * S_inv_x / np.sqrt(bar_x @ S_inv_x)
    else:
        hat_b = np.sqrt((sigma0**2 * ones @ S_inv_1 - 1) / ((bar_x @ S_inv_x) * (ones @ S_inv_1) - (ones @ S_inv_x)**2))
        hat_c = S_inv_1 / (ones @ S_inv_1) + hat_b * (S_inv_x - (ones @ S_inv_x)/(ones @ S_inv_1) * S_inv_1)

    hat_R = hat_c @ bar_x
    hat_r = hat_c @ (hat_Sigma @ hat_c)

    return bar_x, hat_Sigma, hat_c, hat_R, hat_r

spectral decomposition

In [ ]:
np.random.seed(1234)

for n in [500, 1000]:
    for p in [5, 10, 20, 50, 100, 200, 400]:
        dates = lst_date[-n:]
        stocks = np.random.choice(lst_stock, p, replace=False)
        df_new = df.swaplevel().loc[stocks].swaplevel().loc[dates]['return']
        x = np.array(df_new).reshape(p, n)

        eig_vals, eig_vecs = np.linalg.eig(np.corrcoef(x))
        idx = eig_vals.argsort()[::-1]
        eig_vals, eig_vecs = eig_vals[idx], eig_vecs[:, idx]

        plt.scatter(range(len(eig_vals)), eig_vals, s=10)
        plt.axvline(p/20, c='orange', linestyle='--')
        plt.xlabel('n={}, p={}'.format(n, p))
        plt.ylabel('value')
        # plt.show()
        plt.savefig('fig/eig-n{}-p{}.png'.format(n, p))
        plt.cla()

In [ ]:
np.random.seed(1234)
alpha_dict = {}
eig_vals_dict = {}
n = 500
for p in [50, 200, 400]:
    dates = lst_date[-n:]
    stocks = np.random.choice(lst_stock, p, replace=False)
    df_new = df.swaplevel().loc[stocks].swaplevel().loc[dates]['return']
    x = np.array(df_new).reshape(p, n)

    eig_vals, eig_vecs = np.linalg.eig(np.corrcoef(x))
    idx = eig_vals.argsort()[::-1]
    eig_vals, eig_vecs = eig_vals[idx], eig_vecs[:, idx]
    eig_vals_dict[p] = eig_vals

    alpha_dict[p] = []
    for i in range(31): # number of spikes
        alpha_dict[p].append(spec_lse(n, p, eig_vals[i:]))

In [ ]:
for p in alpha_dict.keys():
    plt.plot(np.arange(31), alpha_dict[p], marker='o', markersize=4)
plt.legend(['p={}'.format(p) for p in alpha_dict.keys()])
plt.xlabel('number of deleted eigenvalues')
plt.ylabel('estimated alpha')
plt.show()

In [ ]:
print(alpha_dict[50][50//20], alpha_dict[200][200//20], alpha_dict[400][400//20])

In [ ]:
vals = eig_vals_dict[400][400//20:]
axis = np.linspace(0, np.max(vals), 100)

kde = KernelDensity(kernel='gaussian', bandwidth=0.04).fit(vals.reshape(-1, 1))
logprob = kde.score_samples(axis.reshape(-1, 1))
plt.plot(axis, np.exp(logprob))
plt.plot(axis, lsd_icd(n, 400, axis, alpha_dict[400][400//20]), linestyle='--')
plt.plot(axis, lsd_icd(n, 400, axis, 0.99), linestyle='--')
plt.plot(axis, np.zeros(100), c='black', linestyle='--')
plt.plot(vals, np.full_like(vals, -0.01), '|k', markeredgewidth=1)
plt.legend(['empirical', 'fitted', 'M-P'])
plt.xlabel('n=500, p=400')
plt.ylabel('density')
plt.show()

In [ ]:
for p in alpha_dict.keys():
    vals = eig_vals_dict[p][p//20:]
    axis = np.linspace(0, np.max(vals), 100)

    kde = KernelDensity(kernel='gaussian', bandwidth=0.04).fit(vals.reshape(-1, 1))
    logprob = kde.score_samples(axis.reshape(-1, 1))
    plt.plot(axis, np.exp(logprob))
    plt.plot(axis, icd(axis, alpha_dict[p][p//20]), linestyle='--')
    plt.plot(axis, np.zeros(100), c='black', linestyle='--')
    plt.plot(vals, np.full_like(vals, -0.01), '|k', markeredgewidth=1)
    plt.legend(['ESD', 'PSD'])
    plt.xlabel('n=500, p={}'.format(p))
    plt.ylabel('density')
    plt.show()

Markowitz

In [ ]:
info = np.zeros((45, 2, 8)) # p*quantity*method
n = 500
for i, p in enumerate(np.linspace(10, 450, 45)):
    _, _, _, Rp, rp = equal(n, int(p), 1)
    info[i, :, 0] = [Rp, rp]
    _, _, _, Rp, rp = plugin(n, int(p), 1)
    info[i, :, 1] = [Rp, rp]
    _, _, cp, Rp, rp = bootstrap(n, int(p), 1, repeats=1)
    info[i, :, 2] = [Rp, rp]
    _, _, cp, Rp, rp = bootstrap(n, int(p), 1, repeats=10)
    info[i, :, 3] = [Rp, rp]
    _, _, cp, Rp, rp = bootstrap(n, int(p), 1, repeats=30)
    info[i, :, 4] = [Rp, rp]
    _, _, cp, Rp, rp = spectrum(n, int(p), 1, nspike=int(p//20))
    info[i, :, 5] = [Rp, rp]
    _, _, cp, Rp, rp = spectrum(n, int(p), 1, alpha=0)
    info[i, :, 6] = [Rp, rp]
    _, _, cp, Rp, rp = spectrum(n, int(p), 1, alpha=0.99)
    info[i, :, 7] = [Rp, rp]
    
    if (i + 1) % 5 == 0:
        print(i + 1)

In [ ]:
np.save('info.npy', info)

In [ ]:
# bootstrap repeats comparison
for i in range(2, 5):
    plt.figure(figsize=(4,8),dpi=80)
    plt.plot(np.linspace(10, 450, 45), info[:, 0, 1])
    plt.plot(np.linspace(10, 450, 45), info[:, 0, i], linestyle='--')
    plt.legend(['plug-in', 'bootstrap'])
    plt.xlabel('p')
    plt.ylabel('estimated optimal return')
    plt.show()

In [ ]:
# spectrum comparison
plt.plot(np.linspace(10, 450, 45), info[:, 0, 1])
for i in range(5, 8):
    plt.plot(np.linspace(10, 450, 45), info[:, 0, i], linestyle='--')

plt.legend(['plug-in', 'spectrum fitted', 'spectrum 0', 'spectrum 1 (M-P)'])
plt.xlabel('p')
plt.ylabel('estimated optimal return')
plt.show()

In [ ]:
# # return, R
# for i in range(8):
#     plt.plot(np.linspace(10, 450, 45), info[:, 0, i])
# plt.legend(['equal', 'plug-in', 'bootstrap 1', 'bootstrap 10', 'bootstrap 30', 'spectrum fitted', 'spectrum 0', 'spectrum 1'])
# plt.xlabel('p')
# plt.ylabel('estimated optimal return')
# plt.show()

In [ ]:
# # variance, r
# for i in range(8):
#     plt.plot(np.linspace(10, 450, 45), info[:, 1, i])
# plt.legend(['equal', 'plug-in', 'bootstrap 1', 'bootstrap 10', 'bootstrap 30', 'spectrum fitted', 'spectrum 0', 'spectrum 1'])
# plt.xlabel('p')
# plt.ylabel('estimated variance')
# plt.show()

In [ ]:
@ray.remote
def frep(rep):
    info = np.zeros((4, 3)) # p*method
    n = 500
    for i, p in enumerate([50, 100, 200, 400]):
        _, _, _, info[i, 0], _ = plugin(n, int(p), 1, seed=rep)
        _, _, _, info[i, 1], _ = bootstrap(n, int(p), 1, repeats=30, seed=rep)
        _, _, _, info[i, 2], _ = spectrum(n, int(p), 1, nspike=int(p//20), seed=rep)
        print('rep={}, p={}'.format(rep, p))
    
    return info

In [ ]:
future = [frep.remote(rep) for rep in range(200)]
info2 = ray.get(future)

In [ ]:
info2 = np.array(info2)
np.save('info2-rep200-boot30.npy', info2)

In [ ]:
# info2 = np.load('info2-rep200-boot30.npy')
for i in range(4):
    plt.figure(figsize=(16, 4), dpi=80)
    plt.plot(np.arange(200), info2[:200, i, 0])
    plt.plot(np.arange(200), info2[:200, i, 1])
    plt.plot(np.arange(200), info2[:200, i, 2])
    plt.legend(['plug-in', 'bootstrap', 'spectrum'])
    plt.xlabel('repeats')
    plt.ylabel('estimated optimal return')
    plt.show()

In [ ]:
@ray.remote
def frep_c(rep):
    Rs = np.zeros((4, 3)) # p*method
    cs = np.zeros((4, 3, 400))
    n = 500
    for i, p in enumerate([50, 100, 200, 400]):
        _, _, cs[i, 0, :p], Rs[i, 0], _ = plugin(n, int(p), 1, seed=rep)
        _, _, cs[i, 1, :p], Rs[i, 1], _ = bootstrap(n, int(p), 1, repeats=30, seed=rep)
        _, _, cs[i, 2, :p], Rs[i, 2], _ = spectrum(n, int(p), 1, nspike=int(p//20), seed=rep)
        print('rep={}, p={}'.format(rep, p))
    
    return Rs, cs

In [ ]:
future = frep_c.remote(1234)
info3 = ray.get(future)

In [ ]:
for i, p in enumerate([50, 100, 200, 400]):
    plt.figure(figsize=(16, 4), dpi=80)
    plt.hist(info3[1][i, 0, :p], bins=30, alpha=0.9)
    plt.hist(info3[1][i, 1, :p], bins=30, alpha=0.9)
    plt.hist(info3[1][i, 2, :p], bins=30, alpha=0.9)
    plt.legend(['plug-in', 'bootstrap', 'spectrum'])
    plt.xlabel('estimated optimal allocation')
    plt.ylabel('frequency')
    plt.show()

backtesting

In [ ]:
@ray.remote
def backtest_equal(n, p, sigma0, end_date, seed=1234):
    if seed != None:
        np.random.seed(seed)
    dates = lst_date[-n-end_date-1:-end_date-1]
    df_new = df.loc[dates].swaplevel().loc[stocks].swaplevel()['return']
    
    x = np.array(df_new).reshape(p, n) # p*n array
    bar_x = x.mean(axis=1)
    demean_x = x - bar_x.reshape(p, 1)
    bar_S = demean_x @ demean_x.T / n

    hat_c = np.ones(p) / p
    hat_R = hat_c @ bar_x
    hat_r = hat_c @ (bar_S @ hat_c)

    print('equal: end_date={}'.format(end_date))
    return bar_x, bar_S, hat_c, hat_R, hat_r, end_date

In [ ]:
@ray.remote
def backtest_plugin(n, p, sigma0, end_date, x=None, seed=1234):
    if seed != None:
        np.random.seed(seed)
    dates = lst_date[-n-end_date-1:-end_date-1]
    df_new = df.loc[dates].swaplevel().loc[stocks].swaplevel()['return']

    if type(x) != np.ndarray: # x can be given as bootstrap samples
        x = np.array(df_new).reshape(p, n) # p*n array
    bar_x = x.mean(axis=1)
    demean_x = x - bar_x.reshape(p, 1)
    bar_S = demean_x @ demean_x.T / n
    
    ones = np.ones(p)
    S_inv_x = np.linalg.solve(bar_S, bar_x)
    S_inv_1 = np.linalg.solve(bar_S, ones)
    if sigma0 * ones @ S_inv_x < np.sqrt(bar_x @ S_inv_x):
        hat_c = sigma0 * S_inv_x / np.sqrt(bar_x @ S_inv_x)
    else:
        hat_b = np.sqrt((sigma0**2 * ones @ S_inv_1 - 1) / ((bar_x @ S_inv_x) * (ones @ S_inv_1) - (ones @ S_inv_x)**2))
        hat_c = S_inv_1 / (ones @ S_inv_1) + hat_b * (S_inv_x - (ones @ S_inv_x)/(ones @ S_inv_1) * S_inv_1)

    hat_R = hat_c @ bar_x
    hat_r = hat_c @ (bar_S @ hat_c)

    # print('plugin: end_date={}'.format(end_date))
    return bar_x, bar_S, hat_c, hat_R, hat_r, end_date

In [ ]:
@ray.remote
def backtest_bootstrap(n, p, sigma0, end_date, repeats=30, seed=1234):
    bar_x, bar_S, cp, Rp, _, _ = ray.get(backtest_plugin.remote(n, p, sigma0, end_date)) # plug-in estimator
    
    cs, Rs = [], []
    for _ in range(repeats):
        bsample = np.random.multivariate_normal(mean=bar_x, cov=bar_S, size=n).T # p*n 
        _, _, cb, Rb, _, _ = ray.get(backtest_plugin.remote(n, p, sigma0, end_date, x=bsample))
        cs.append(cb)
        Rs.append(Rb)
    
    gamma = 1 / (1 - p / n)
    hat_c = cp + (cp - np.array(cs).mean(axis=0)) / np.sqrt(gamma)
    hat_R = Rp + (Rp - np.mean(Rs)) / np.sqrt(gamma)
    hat_r = hat_c @ (bar_S @ hat_c)

    print('bootstrap: end_date={}'.format(end_date))
    return bar_x, bar_S, hat_c, hat_R, hat_r, end_date

In [ ]:
@ray.remote
def backtest_spectrum(n, p, sigma0, end_date, method='lse', nspike=0, alpha=None, seed=1234):
    bar_x, bar_S, cp, Rp, _, _ = ray.get(backtest_plugin.remote(n, p, sigma0, end_date, seed=seed)) # plug-in estimator

    # spectral decomposition
    std = np.sqrt(np.diag(bar_S))
    corr = np.diag(1 / std) @ bar_S @ np.diag(1 / std)
    eig_vals, eig_vecs = np.linalg.eig(corr)
    idx = eig_vals.argsort() # from small to large
    eig_vals, eig_vecs = eig_vals[idx], eig_vecs[:, idx]

    # spectrum correction
    if method == 'lse': # use inverse cubic density
        if alpha == None:
            alpha = spec_lse(n, p, eig_vals) if nspike == 0 else spec_lse(n, p, eig_vals[:-nspike])
        # print(alpha)
        spec = icd_qtile(np.linspace(1 / 2 / p, 1 - 1 / 2 / p, p), alpha)
    else:
        raise NotImplementedError
    
    hat_Sigma = np.diag(std) @ eig_vecs @ np.diag(spec) @ eig_vecs.T @ np.diag(std) # corrected estimate of covariance

    ones = np.ones(p)
    S_inv_x = np.linalg.solve(hat_Sigma, bar_x)
    S_inv_1 = np.linalg.solve(hat_Sigma, ones)
    if sigma0 * ones @ S_inv_x < np.sqrt(bar_x @ S_inv_x):
        hat_c = sigma0 * S_inv_x / np.sqrt(bar_x @ S_inv_x)
    else:
        hat_b = np.sqrt((sigma0**2 * ones @ S_inv_1 - 1) / ((bar_x @ S_inv_x) * (ones @ S_inv_1) - (ones @ S_inv_x)**2))
        hat_c = S_inv_1 / (ones @ S_inv_1) + hat_b * (S_inv_x - (ones @ S_inv_x)/(ones @ S_inv_1) * S_inv_1)

    hat_R = hat_c @ bar_x
    hat_r = hat_c @ (hat_Sigma @ hat_c)

    print('spectrum: end_date={}'.format(end_date))
    return bar_x, hat_Sigma, hat_c, hat_R, hat_r, end_date

In [ ]:
nbacktest = 500
n = 500
p = 400

np.random.seed(1234)
stocks = np.random.choice(lst_stock, p, replace=False)
df_new = df.swaplevel().loc[stocks].swaplevel()

In [ ]:
future_e = [backtest_equal.remote(n, p, 1, end_date) for end_date in range(nbacktest)]
info_e = ray.get(future_e)
info_e.sort(key=lambda x: x[-1])

ce = np.array([info_e[i][2] for i in range(nbacktest)])
Re = np.array([info_e[i][3] for i in range(nbacktest)])
re = np.array([info_e[i][4] for i in range(nbacktest)])

In [ ]:
np.save('backtest/ce.npy', ce)
np.save('backtest/Re.npy', Re)
np.save('backtest/re.npy', re)

In [ ]:
future_p = [backtest_plugin.remote(n, p, 1, end_date) for end_date in range(nbacktest)]
info_p = ray.get(future_p)
info_p.sort(key=lambda x: x[-1])

cp = np.array([info_p[i][2] for i in range(nbacktest)])
Rp = np.array([info_p[i][3] for i in range(nbacktest)])
rp = np.array([info_p[i][4] for i in range(nbacktest)])

In [ ]:
np.save('backtest/cp.npy', cp)
np.save('backtest/Rp.npy', Rp)
np.save('backtest/rp.npy', rp)

In [ ]:
future_s = [backtest_spectrum.remote(n, p, 1, end_date, nspike=int(p//20)) for end_date in range(nbacktest)]
info_s = ray.get(future_s)
info_s.sort(key=lambda x: x[-1])

cs = np.array([info_s[i][2] for i in range(nbacktest)])
Rs = np.array([info_s[i][3] for i in range(nbacktest)])
rs = np.array([info_s[i][4] for i in range(nbacktest)])

In [ ]:
np.save('backtest/cs.npy', cs)
np.save('backtest/Rs.npy', Rs)
np.save('backtest/rs.npy', rs)

In [ ]:
future_b = [backtest_bootstrap.remote(n, p, 1, end_date, repeats=30) for end_date in range(nbacktest)]
info_b = ray.get(future_b)
info_b.sort(key=lambda x: x[-1])

cb = np.array([info_b[i][2] for i in range(nbacktest)])
Rb = np.array([info_b[i][3] for i in range(nbacktest)])
rb = np.array([info_b[i][4] for i in range(nbacktest)])

In [ ]:
np.save('backtest/cb.npy', cb)
np.save('backtest/Rb.npy', Rb)
np.save('backtest/rb.npy', rb)

In [ ]:
dates = lst_date[-n-1:-1] # in sample
# dates = lst_date[-n:] # out of sample
ret = df.loc[dates].swaplevel().loc[stocks].swaplevel()['return']
ret = np.array(ret).reshape(p, n)

In [ ]:
plt.figure(figsize=(16, 4), dpi=80)
plt.plot((cp[::-1, :] * ret.T).mean(axis=1).cumsum())
plt.plot((cb[::-1, :] * ret.T).mean(axis=1).cumsum())
plt.plot((cs[::-1, :] * ret.T).mean(axis=1).cumsum())
plt.plot(np.zeros(n), c='black', linestyle='--')
plt.legend(['plug-in', 'bootstrap', 'spectrum'])
plt.xlabel('trading date')
plt.ylabel('cumulative return')
plt.show()

In [ ]:
cp2 = cp[::-1, :].clip(0)
cp2 = cp2 / cp2.sum(axis=1, keepdims=True)
cb2 = cb[::-1, :].clip(0)
cb2 = cb2 / cb2.sum(axis=1, keepdims=True)
cs2 = cs[::-1, :].clip(0)
cs2 = cs2 / cs2.sum(axis=1, keepdims=True)

In [ ]:
plt.figure(figsize=(16, 4), dpi=80)
plt.plot((cp2 * ret.T).mean(axis=1).cumsum())
plt.plot((cb2 * ret.T).mean(axis=1).cumsum())
plt.plot((cs2 * ret.T).mean(axis=1).cumsum())
plt.plot((ce * ret.T).mean(axis=1).cumsum())
plt.plot(np.zeros(n), c='black', linestyle='--')
plt.legend(['plug-in', 'bootstrap', 'spectrum', 'equal'])
plt.xlabel('trading date')
plt.ylabel('cumulative return')
plt.show()

In [ ]:
plt.figure(figsize=(16, 4), dpi=80)
plt.plot((cp2 * ret.T).mean(axis=1).cumsum() - (ce * ret.T).mean(axis=1).cumsum())
plt.plot((cb2 * ret.T).mean(axis=1).cumsum() - (ce * ret.T).mean(axis=1).cumsum())
plt.plot((cs2 * ret.T).mean(axis=1).cumsum() - (ce * ret.T).mean(axis=1).cumsum())
plt.plot(np.zeros(n), c='black', linestyle='--')
plt.legend(['plug-in', 'bootstrap', 'spectrum'])
plt.xlabel('trading date')
plt.ylabel('cumulative return')
plt.show()